In [1]:
!pip install --quiet datasets transformers evaluate huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.7 MB/s eta 0:00:00


In [2]:
%pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pla

In [3]:
from datasets import load_dataset

# Download & cache into Colab’s local disk
dataset = load_dataset("ag_news", download_mode="force_redownload")

train_ds = dataset["train"]
test_ds  = dataset["test"]

# Quick peek
print(train_ds[0])
print(train_ds.features)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'label': 2}
{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['World', 'Sports', 'Business', 'Sci/Tech'], id=None)}


In [4]:
import pandas as pd


In [5]:
# turn to DataFrame
news_df = pd.DataFrame(train_ds)

# counting how many examples per label
counts = news_df["label"].value_counts().sort_index()
print(counts)

label_names = train_ds.features["label"].names
print({label: label_names[label] for label in counts.index})

label
0    30000
1    30000
2    30000
3    30000
Name: count, dtype: int64
{0: 'World', 1: 'Sports', 2: 'Business', 3: 'Sci/Tech'}


In [6]:
news_df


,text,label
0,Wall St. Bears Claw Back Into the Black (Reute...,2
1,Carlyle Looks Toward Commercial Aerospace (Reu...,2
2,Oil and Economy Cloud Stocks' Outlook (Reuters...,2
3,Iraq Halts Oil Exports from Main Southern Pipe...,2
4,"Oil prices soar to all-time record, posing new...",2
...,...,...
119995,Pakistan's Musharraf Says Won't Quit as Army C...,0
119996,Renteria signing a top-shelf deal Red Sox gene...,1
119997,Saban not going to Dolphins yet The Miami Dolp...,1
119998,Today's NFL games PITTSBURGH at NY GIANTS Time...,1


In [7]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [8]:
!pip install --quiet nltk

In [9]:
import nltk
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("omw-1.4")
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [10]:
import re
import string
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))


def preprocess_text_simple(text):
    # 1. Lowercase
    text = text.lower()
    # 2. Remove digits
    text = re.sub(r"\d+", "", text)
    # 3. Strip punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    # 4. Split on whitespace
    tokens = text.split()
    # 5. Remove stopwords
    tokens = [t for t in tokens if t not in stop_words]
    # 6. Re-join
    return " ".join(tokens)

# Testing
import pandas as pd
sample = pd.DataFrame(train_ds.select(range(5)))
sample["cleaned"] = sample["text"].apply(preprocess_text_simple)
print(sample[["text","cleaned"]])

                                                text  \
0  Wall St. Bears Claw Back Into the Black (Reute...   
1  Carlyle Looks Toward Commercial Aerospace (Reu...   
2  Oil and Economy Cloud Stocks' Outlook (Reuters...   
3  Iraq Halts Oil Exports from Main Southern Pipe...   
4  Oil prices soar to all-time record, posing new...   

                                             cleaned  
0  wall st bears claw back black reuters reuters ...  
1  carlyle looks toward commercial aerospace reut...  
2  oil economy cloud stocks outlook reuters reute...  
3  iraq halts oil exports main southern pipeline ...  
4  oil prices soar alltime record posing new mena...  


In [11]:
news_df["cleaned"] = news_df["text"].apply(preprocess_text_simple)


print(news_df[["text","cleaned"]].head())

                                                text  \
0  Wall St. Bears Claw Back Into the Black (Reute...   
1  Carlyle Looks Toward Commercial Aerospace (Reu...   
2  Oil and Economy Cloud Stocks' Outlook (Reuters...   
3  Iraq Halts Oil Exports from Main Southern Pipe...   
4  Oil prices soar to all-time record, posing new...   

                                             cleaned  
0  wall st bears claw back black reuters reuters ...  
1  carlyle looks toward commercial aerospace reut...  
2  oil economy cloud stocks outlook reuters reute...  
3  iraq halts oil exports main southern pipeline ...  
4  oil prices soar alltime record posing new mena...  
